# Quick-start with BoFire

## Install BoFire

Run the following command to install BoFire and its optimization feature:

```
pip install bofire[optimization]
```

This command will automatically install BoTorch, which is a dependency of BoFire, as well as the optimization feature.

## Single objective BO

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import time

from bofire.data_models.features.api import (
    ContinuousInput,
    ContinuousOutput,
)

from IPython.display import clear_output

import bofire.strategies.api as strategies
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.acquisition_functions.api import qEI

from bofire.data_models.domain.api import Domain

from bofire.data_models.domain.api import Domain, Inputs, Outputs

from bofire.data_models.objectives.api import MinimizeObjective

C:\Users\chrihaas\Documents\git\bofire\bofire\surrogates\xgb.py:12: UserWarning: xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.
  warnings.warn("xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.")


Explore Himmelblau optimization function

In mathematical optimization, Himmelblau's function is a multi-modal function, used to test the performance of optimization algorithms.

In [2]:
def himmelblau(x_1, x_2):
    return ((x_1**2 + x_2 - 11)**2+(x_1 + x_2**2 -7)**2)

Explore the function, which has four minima

In [3]:
x_1_vec = np.linspace(-6, 6, 100)
x_2_vec = np.linspace(-6, 6, 100)

x_1, x_2 = np.meshgrid(x_1_vec, x_2_vec)
results = himmelblau(x_1, x_2)

x_minima = [
        [3.0, 2.0],
        [-2.805118, 3.131312],
        [-3.779310, -3.283186],
        [3.584428, -1.848126],
    ]
y_minima = np.zeros(4)


Contour plot

In [4]:
fig = go.Figure(data=go.Contour(z=results, x=x_1_vec, y=x_2_vec, contours_coloring="heatmap"))
fig.update_layout(
    title='Himmelblau Function',
    autosize=False,
    width=750,
    height=750,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
        xaxis_title='x_1',
        yaxis_title='x_2',
        zaxis_title='y',
    ))
for i in range(4):
    fig.add_trace(go.Scatter(x=[x_minima[i][0]], y=[x_minima[i][1]],
                             mode='markers', showlegend=False,
                             marker=dict(size=10, color='red')))
fig.show()

3D plot

In [5]:
fig = go.Figure(data=go.Surface(z=results, x=x_1_vec, y=x_2_vec))
fig.update_layout(
    title='Himmelblau Function',
    autosize=False,
    width=750,
    height=750,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
        xaxis_title='x_1',
        yaxis_title='x_2',
        zaxis_title='y',
    ))
for i in range(4):
    fig.add_trace(go.Scatter3d(x=[x_minima[i][0]], y=[x_minima[i][1]], z=[y_minima[i]],
                             mode='markers', showlegend=False,
                             marker=dict(size=10, color='red')))
fig.show()

Define optimization domain

In [8]:
input_feature_1 = ContinuousInput(key="x_1", bounds=(-6, 6))
input_feature_2 = ContinuousInput(key="x_2", bounds=(-6, 6))

objective = MinimizeObjective(w=1.0)

output_feature = ContinuousOutput(key="y", objective=objective)

domain = Domain(
    inputs=Inputs(features=[input_feature_1, input_feature_2]),
    outputs=Outputs(features=[output_feature]),
)


Define function to plot model mean prediction and samples

In [9]:
def plot_himmelblau_opt(samples, model_prediction):
    fig = go.Figure(data=go.Contour(z=model_prediction, x=x_1_vec, y=x_2_vec, contours_coloring="heatmap"))
    fig.update_layout(
        title=f'Model prediction, best so far: {round(samples["y"].min(), 2)}',
        autosize=False,
        width=750,
        height=750,
        margin=dict(l=65, r=50, b=65, t=90),
        scene=dict(
            xaxis_title='x_1',
            yaxis_title='x_2',
            zaxis_title='y',
        ))
    for i in range(4):
       fig.add_trace(go.Scatter(x=[x_minima[i][0]], y=[x_minima[i][1]],
                               mode='markers', showlegend=False,
                               marker=dict(size=10, color='red')))
    fig.add_trace(go.Scatter(x=samples["x_1"], y=samples["x_2"], mode='lines', showlegend=False, line=dict(color='blue', width=2)))
    return fig

Run optimization

In [10]:
# initialize
samples = domain.inputs.sample(10, seed=19)
y = []
for i, row in samples.iterrows():
    y.append(himmelblau(row["x_1"], row["x_2"]))
samples["y"] = y

sobo_strategy_data_model = SoboStrategy(domain=domain, acquisition_function=qEI(), seed=19)

sobo_strategy = strategies.map(sobo_strategy_data_model)

sobo_strategy.tell(experiments=samples)

prediction = sobo_strategy.predict(pd.DataFrame({"x_1": x_1.flatten(), "x_2": x_2.flatten()}))
pred = np.array(prediction["y_pred"]).reshape(100, 100)
fig = plot_himmelblau_opt(samples, pred)
fig.show()

# optimize
for i in range(30):
    new_sample = sobo_strategy.ask(candidate_count=1)

    y = []
    for i, row in new_sample.iterrows():
        y.append(himmelblau(row["x_1"], row["x_2"]))

    new_sample["y"] = y
    samples = pd.concat([samples, new_sample], join="inner")

    sobo_strategy.tell(experiments=samples)
    
    clear_output(wait=True)
    prediction = sobo_strategy.predict(pd.DataFrame({"x_1": x_1.flatten(), "x_2": x_2.flatten()}))
    pred = np.array(prediction["y_pred"]).reshape(100, 100)
    fig = plot_himmelblau_opt(samples, pred)
    fig.show()

    time.sleep(0.1)

    

Additional comments/ideas:

Should we visualize (e.g., euclidian) distance to all four minima points (four line plots over iterations)

Parallel coordinates plot to see the four minima places

Plot also model variance, acquisition function